# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [4]:
import pandas as pd
import numpy as np
import statsmodels as sm
from statsmodels.api import formula as smf

In [5]:
df = pd.read_csv('previsao_de_renda.csv')
df.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [19]:
print(df.sexo.value_counts())
print(df.tipo_renda.value_counts())
print(df.educacao.value_counts())
print(df.estado_civil.value_counts())
print(df.tipo_residencia.value_counts())

F    10119
M     4881
Name: sexo, dtype: int64
Assalariado         7633
Empresário          3508
Pensionista         2582
Servidor público    1268
Bolsista               9
Name: tipo_renda, dtype: int64
Secundário             8895
Superior completo      5335
Superior incompleto     579
Primário                165
Pós graduação            26
Name: educacao, dtype: int64
Casado      10534
Solteiro     1798
União        1078
Separado      879
Viúvo         711
Name: estado_civil, dtype: int64
Casa             13532
Com os pais        675
Governamental      452
Aluguel            194
Estúdio             83
Comunitário         64
Name: tipo_residencia, dtype: int64


In [23]:
reg = smf.ols('''np.log(renda) ~ C(sexo, Treatment('F')) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda, Treatment('Assalariado'))
              + C(educacao, Treatment('Secundário')) + C(estado_civil, Treatment('Casado')) + C(tipo_residencia, Treatment('Casa'))
              + idade + tempo_emprego + qt_pessoas_residencia''', data = df).fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Tue, 09 Apr 2024   Prob (F-statistic):               0.00
Time:                        14:18:10   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.5264      0.219     29.853      0.000       6.098       6.955
C(sexo, Treatment('F'))[T.M]                                    0.7874      0.015     53.723      0.000       0.759       0.816
posse_de_veiculo[T.True]                                        0.0441      0.014      3.119      0.002       0.016       0.072
posse_de_imovel[T.True]                                         0.0829      0.014      5.926      0.000       0.055       0.110
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment('Secundário'))[T.Primário]                0.0141      0.072      0.196      0.844      -0.127       0.155
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1212      0.142      0.853      0.394      -0.157       0.400
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1079      0.014      7.763      0.000       0.081       0.135
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0290      0.032     -0.900      0.368      -0.092       0.034
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil, Treatment('Casado'))[T.União]                  -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil, Treatment('Casado'))[T.Viúvo]                   0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment('Casa'))[T.Aluguel]                0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment('Casa'))[T.Com os pais]            0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment('Casa'))[T.Comunitário]           -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment('Casa'))[T.Estúdio]                0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment('Casa'))[T.Governamental]         -0.0129      0.039     -0.331      0.741      -0.089       0.064
qtd_filhos 

In [24]:
reg = smf.ols('''np.log(renda) ~ C(sexo, Treatment('F')) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda, Treatment('Assalariado'))
              + C(educacao, Treatment('Secundário')) + C(estado_civil, Treatment('Casado'))
              + idade + tempo_emprego + qt_pessoas_residencia''', data = df).fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Tue, 09 Apr 2024   Prob (F-statistic):               0.00
Time:                        14:19:47   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.5315      0.218     29.894      0.000       6.103       6.960
C(sexo, Treatment('F'))[T.M]                                    0.7893      0.015     53.999      0.000       0.761       0.818
posse_de_veiculo[T.True]                                        0.0442      0.014      3.125      0.002       0.016       0.072
posse_de_imovel[T.True]                                         0.0819      0.014      5.966      0.000       0.055       0.109
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2219      0.241      0.920      0.357      -0.251       0.695
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1556      0.015     10.433      0.000       0.126       0.185
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.3116      0.241     -1.292      0.196      -0.784       0.161
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0589      0.022      2.653      0.008       0.015       0.102
C(educacao, Treatment('Secundário'))[T.Primário]                0.0097      0.072      0.135      0.893      -0.131       0.150
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1200      0.142      0.845      0.398      -0.159       0.399
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1077      0.014      7.761      0.000       0.080       0.135
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0288      0.032     -0.893      0.372      -0.092       0.034
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3272      0.111      2.942      0.003       0.109       0.545
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2683      0.109      2.464      0.014       0.055       0.482
C(estado_civil, Treatment('Casado'))[T.União]                  -0.0329      0.025     -1.310      0.190      -0.082       0.016
C(estado_civil, Treatment('Casado'))[T.Viúvo]                   0.3678      0.116      3.182      0.001       0.141       0.594
qtd_filhos                                                     -0.2655      0.108     -2.460      0.014      -0.477      -0.054
idade                                                           0.0052      0.001      6.579      0.000       0.004       0.007
tempo_emprego                                                   0.0617      0.001     59.485      0.000       0.060       0.064
qt_pessoas_residencia                                           0.2923      0.108      2.713      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.825   Durbin-Watson:      

*Retirei a variável 'tipo_residencia'. O R² não teve alteração mas o AIC reduziu um pouco*

In [36]:
df = df[df['educacao'] != 'Primário']
df = df[df['educacao'] != 'Pós graduação']
df = df[df['educacao'] != 'Superior incompleto']

In [40]:
reg = smf.ols('''np.log(renda) ~ C(sexo, Treatment('F')) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda, Treatment('Assalariado'))
              + C(educacao, Treatment('Secundário')) + C(estado_civil, Treatment('Casado'))
              + idade + tempo_emprego + qt_pessoas_residencia''', data = df).fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.360
Method:                 Least Squares   F-statistic:                     413.0
Date:                Tue, 09 Apr 2024   Prob (F-statistic):               0.00
Time:                        14:39:11   Log-Likelihood:                -12842.
No. Observations:               11740   AIC:                         2.572e+04
Df Residuals:                   11723   BIC:                         2.584e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.5417      0.219     29.863      0.000       6.112       6.971
C(sexo, Treatment('F'))[T.M]                                    0.7983      0.015     52.804      0.000       0.769       0.828
posse_de_veiculo[T.True]                                        0.0447      0.015      3.056      0.002       0.016       0.073
posse_de_imovel[T.True]                                         0.0810      0.014      5.713      0.000       0.053       0.109
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2235      0.242      0.925      0.355      -0.250       0.697
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1570      0.015     10.201      0.000       0.127       0.187
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.3135      0.242     -1.297      0.195      -0.787       0.160
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0578      0.023      2.554      0.011       0.013       0.102
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1077      0.014      7.742      0.000       0.080       0.135
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3123      0.112      2.800      0.005       0.094       0.531
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2669      0.109      2.445      0.015       0.053       0.481
C(estado_civil, Treatment('Casado'))[T.União]                  -0.0389      0.026     -1.490      0.136      -0.090       0.012
C(estado_civil, Treatment('Casado'))[T.Viúvo]                   0.3648      0.116      3.144      0.002       0.137       0.592
qtd_filhos                                                     -0.2709      0.108     -2.505      0.012      -0.483      -0.059
idade                                                           0.0050      0.001      6.127      0.000       0.003       0.007
tempo_emprego                                                   0.0618      0.001     58.241      0.000       0.060       0.064
qt_pessoas_residencia                                           0.2921      0.108      2.706      0.007       0.080       0.504
==============================================================================
Omnibus:                        0.834   Durbin-Watson:                   2.009
Prob(Omnibus):                  0.659   Jarque-Bera (JB):                0.821
Skew:                           0.020   Prob(JB):                        0.663
Kurtosis:                       3.007   Cond. No.                     2.09e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covar

*Retirei as categorias da variável 'educação' que eram irrelevantes e o modelo ficou um pouco melhor*

In [41]:
df = df[df['tipo_renda'] != 'Bolsista']
df = df[df['tipo_renda'] != 'Pensionista']
df = df[df['estado_civil'] != 'União']

In [42]:
reg = smf.ols('''np.log(renda) ~ C(sexo, Treatment('F')) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda, Treatment('Assalariado'))
              + C(educacao, Treatment('Secundário')) + C(estado_civil, Treatment('Casado'))
              + idade + tempo_emprego + qt_pessoas_residencia''', data = df).fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.362
Method:                 Least Squares   F-statistic:                     475.4
Date:                Tue, 09 Apr 2024   Prob (F-statistic):               0.00
Time:                        14:43:09   Log-Likelihood:                -11913.
No. Observations:               10860   AIC:                         2.385e+04
Df Residuals:                   10846   BIC:                         2.396e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.4909      0.230     28.209      0.000       6.040       6.942
C(sexo, Treatment('F'))[T.M]                                    0.8075      0.016     51.335      0.000       0.777       0.838
posse_de_veiculo[T.True]                                        0.0437      0.015      2.878      0.004       0.014       0.073
posse_de_imovel[T.True]                                         0.0800      0.015      5.397      0.000       0.051       0.109
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1624      0.016     10.077      0.000       0.131       0.194
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0562      0.023      2.397      0.017       0.010       0.102
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1053      0.015      7.252      0.000       0.077       0.134
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3360      0.117      2.874      0.004       0.107       0.565
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2903      0.115      2.534      0.011       0.066       0.515
C(estado_civil, Treatment('Casado'))[T.Viúvo]                   0.3902      0.121      3.219      0.001       0.153       0.628
qtd_filhos                                                     -0.2944      0.114     -2.590      0.010      -0.517      -0.072
idade                                                           0.0050      0.001      5.976      0.000       0.003       0.007
tempo_emprego                                                   0.0620      0.001     56.288      0.000       0.060       0.064
qt_pessoas_residencia                                           0.3146      0.113      2.774      0.006       0.092       0.537
==============================================================================
Omnibus:                        0.540   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.763   Jarque-Bera (JB):                0.532
Skew:                           0.017   Prob(JB):                        0.767
Kurtosis:                       3.004   Cond. No.                     2.11e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.11e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Retirei as categorias das variáveis 'tipo_renda' e 'estado_civil' que tambem não eram estatísticamente relevantes chegando a um melhor R², R² ajustado e AIC.